# Load data in HDFS

In [1]:
'''
Uncomment below to copy files to local HDFS.
Assumes that code lives in git directory and that data/ lives one level above
'''
# !hadoop fs -copyFromLocal ../data/movies.csv /
# !hadoop fs -ls /
# !hadoop fs -cat /movies.csv | head

'\nUncomment below to copy files to local HDFS.\nAssumes that code lives in git directory and that data/ lives one level above\n'

# Start Spark Session

In [2]:
# Change the number of cores in this code block
# by setting `spark.master` to `local[n]` where
# n is the number of cores
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

conf = pyspark.SparkConf().setAll([('spark.master', 'local[2]'),
                                   ('spark.app.name', 'Basic Setup')])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-27 22:07:51,908 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load Data

Read data from the `ratings.csv` file

In [3]:
movies_df = spark.read.option("header",True).csv("hdfs:///movies.csv").cache()
movies_df.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [4]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:
genres_df = movies_df.select(split("genres","\|").alias("genres"))
genres_df = genres_df.select(explode("genres").alias("word"))
genres_df.show(5, truncate=False)

+---------+
|word     |
+---------+
|Adventure|
|Animation|
|Children |
|Comedy   |
|Fantasy  |
+---------+
only showing top 5 rows



In [6]:
distinct_genres = genres_df.groupBy('word').count().sort(col("count").desc())
print(f"Total distinct genres: {distinct_genres.count()}")
distinct_genres.show(25, truncate=False)

Total distinct genres: 20
+------------------+-----+
|word              |count|
+------------------+-----+
|Drama             |25606|
|Comedy            |16870|
|Thriller          |8654 |
|Romance           |7719 |
|Action            |7348 |
|Horror            |5989 |
|Documentary       |5605 |
|Crime             |5319 |
|(no genres listed)|5062 |
|Adventure         |4145 |
|Sci-Fi            |3595 |
|Children          |2935 |
|Animation         |2929 |
|Mystery           |2925 |
|Fantasy           |2731 |
|War               |1874 |
|Western           |1399 |
|Musical           |1054 |
|Film-Noir         |353  |
|IMAX              |195  |
+------------------+-----+



In [7]:
spark.stop()